In [1]:
from models import CondEnergyModel, UNet, MyResnet, autoencoders, REDNet30
from ResNet import IGEBM, EnResnet, EnResnetSN
from sn_gan import Discriminator
from utils import permute, to_numpy, init_weights, Self_Energy_log, get_dataloaders
# import torchsummary
from torchinfo import summary
import datetime, random, os



In [3]:
en = UNet(1, 32, 1)
print(summary(en, (10, 1, 182, 182)))

Layer (type:depth-idx)                        Output Shape              Param #
UNet                                          [10, 1, 182, 182]         12,718,592
├─DoubleConv: 1-1                             [10, 64, 182, 182]        --
│    └─Sequential: 2-1                        [10, 64, 182, 182]        --
│    │    └─Conv2d: 3-1                       [10, 64, 182, 182]        576
│    │    └─BatchNorm2d: 3-2                  [10, 64, 182, 182]        128
│    │    └─ReLU: 3-3                         [10, 64, 182, 182]        --
│    │    └─Conv2d: 3-4                       [10, 64, 182, 182]        36,864
│    │    └─BatchNorm2d: 3-5                  [10, 64, 182, 182]        128
│    │    └─ReLU: 3-6                         [10, 64, 182, 182]        --
├─Down: 1-2                                   [10, 128, 91, 91]         --
│    └─Sequential: 2-2                        [10, 128, 91, 91]         --
│    │    └─MaxPool2d: 3-7                    [10, 64, 91, 91]          --
│    

In [3]:
ae = REDNet30()
print(summary(ae, (10, 3, 32, 32)))


Layer (type:depth-idx)                   Output Shape              Param #
REDNet30                                 [10, 3, 32, 32]           --
├─Sequential: 1-1                        --                        --
│    └─Sequential: 2-1                   [10, 64, 16, 16]          --
│    │    └─Conv2d: 3-1                  [10, 64, 16, 16]          1,792
│    │    └─ReLU: 3-2                    [10, 64, 16, 16]          --
│    │    └─BatchNorm2d: 3-3             [10, 64, 16, 16]          128
│    └─Sequential: 2-2                   [10, 64, 16, 16]          --
│    │    └─Conv2d: 3-4                  [10, 64, 16, 16]          36,928
│    │    └─ReLU: 3-5                    [10, 64, 16, 16]          --
│    │    └─BatchNorm2d: 3-6             [10, 64, 16, 16]          128
│    └─Sequential: 2-3                   [10, 64, 16, 16]          --
│    │    └─Conv2d: 3-7                  [10, 64, 16, 16]          36,928
│    │    └─ReLU: 3-8                    [10, 64, 16, 16]          --
│ 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Input, Dense, Reshape, Conv2DTranspose,\
   Activation, BatchNormalization, ReLU, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import  cifar10

2023-07-05 11:26:28.854607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 11:26:28.905881: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 11:26:28.907302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 11:26:29.668522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
import numpy as np
Nsampling = 2000 #exponential schedule with flat region in the beginning and end
Tmax,Tmin = 100,1
T = Tmax*np.exp(-np.linspace(0,Nsampling-1,Nsampling)*(np.log(Tmax/Tmin)/Nsampling))
T = np.concatenate((Tmax*np.ones((500,)),T),axis=0)
T = np.concatenate((T,Tmin*np.linspace(1,0,200)),axis=0)
T

array([1.00000000e+02, 1.00000000e+02, 1.00000000e+02, ...,
       1.00502513e-02, 5.02512563e-03, 0.00000000e+00])

In [3]:
ae = ConvAE(32, 256)
# en = CNN_Encoder(32)
print(summary(ae, (10, 3, 32, 32)))

Layer (type:depth-idx)                   Output Shape              Param #
ConvAE                                   [10, 3, 32, 32]           --
├─Encoder: 1-1                           [10, 256]                 --
│    └─Sequential: 2-1                   [10, 256]                 --
│    │    └─Conv2d: 3-1                  [10, 32, 16, 16]          896
│    │    └─GELU: 3-2                    [10, 32, 16, 16]          --
│    │    └─Conv2d: 3-3                  [10, 32, 16, 16]          9,248
│    │    └─GELU: 3-4                    [10, 32, 16, 16]          --
│    │    └─Conv2d: 3-5                  [10, 64, 8, 8]            18,496
│    │    └─GELU: 3-6                    [10, 64, 8, 8]            --
│    │    └─Conv2d: 3-7                  [10, 64, 8, 8]            36,928
│    │    └─GELU: 3-8                    [10, 64, 8, 8]            --
│    │    └─Conv2d: 3-9                  [10, 64, 4, 4]            36,928
│    │    └─GELU: 3-10                   [10, 64, 4, 4]            --

In [3]:
# en = MyResnet(3,32).cuda()
# en = Discriminator(3, 256, 'swish').cuda()
# en = IGEBM(3, 256).cuda()
en = EnResnet(3, 64, 3).cuda()
size = 32
print(summary(en, (10, 3, size, size)))

Layer (type:depth-idx)                   Output Shape              Param #
EnResnet                                 [10, 3]                   --
├─Sequential: 1-1                        [10, 64, 32, 32]          --
│    └─Conv2d: 2-1                       [10, 64, 32, 32]          1,728
│    └─Swish: 2-2                        [10, 64, 32, 32]          --
│    └─Conv2d: 2-3                       [10, 64, 32, 32]          36,864
│    └─Swish: 2-4                        [10, 64, 32, 32]          --
├─Sequential: 1-2                        [10, 512, 4, 4]           --
│    └─EnResBlock: 2-5                   [10, 64, 32, 32]          --
│    │    └─Sequential: 3-1              [10, 64, 32, 32]          73,728
│    │    └─Sequential: 3-2              [10, 64, 32, 32]          --
│    │    └─Sequential: 3-3              --                        (recursive)
│    └─EnResBlock: 2-6                   [10, 128, 32, 32]         --
│    │    └─Sequential: 3-4              [10, 128, 32, 32]       